In [1]:
import pandas as pd

In [7]:
from bs4 import BeautifulSoup

In [4]:
pd.read_pickle('/home/charubaiel/workflow/projects/MLOpsini/pipelines/.DH/storage/cian_dataframe')

,datetime,title,price,publish_delta,url,id,text,Город,Округ,Метро,Район,Улица,Дом,metro_branch,metro_name,metro_dist,img_list
0,1.682547e+09,Большой выбор! Звоните!,от3500₽/сутки,8 часов назад,https://www.cian.ru/rent/flat/9043033/,9043033,Реальные фото! Отчётные документы!\n\nКрасивая...,Москва,ЦАО,р-н Тверской,м. Белорусская,Приютский переулок,3,,,,[https://cdn-p.cian.site/images/43/014/kvartir...
1,1.682547e+09,Квартира Свободна. ТОП! Звони!,от4000₽/сутки,8 часов назад,https://www.cian.ru/rent/flat/214768370/,214768370,"Приветствую Вас, уважаемый гость! \nБудем рады...",Москва,ЦАО,р-н Тверской,м. Охотный ряд,Тверская улица,6С1,,,,[https://cdn-p.cian.site/images/88/108/831/pos...
2,1.682547e+09,"1-комн. кв., 40 м², 10/17 этаж",от3000₽/сутки,8 часов назад,https://www.cian.ru/rent/flat/272844272/,272844272,"Приветствую Вас, уважаемый гость! \nБудем рады...",Москва,ЦАО,р-н Замоскворечье,м. Новокузнецкая,Озерковская набережная,2/1,,,,[https://cdn-p.cian.site/images/12/686/821/pos...
3,1.682547e+09,Свободна! 3 мин от метро.,от3000₽/сутки,11 часов назад,https://www.cian.ru/rent/flat/268837029/,268837029,Без посредников и дополнительных комиссий . Га...,Москва,СВАО,р-н Отрадное,м. Владыкино,Алтуфьевское шоссе,2,,,,[https://cdn-p.cian.site/images/86/899/021/pos...
4,1.682547e+09,Свежий ремонт! Свободна!,от3500₽/сутки,10 часов назад,https://www.cian.ru/rent/flat/279094765/,279094765,Просторная и светлая 1-комнатная двухуровневая...,Москва,ЮАО,р-н Даниловский,м. Тульская,2-я Рощинская улица,10,,,,[https://cdn-p.cian.site/images/46/276/931/pos...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.682547e+09,Стильная 2к квартира на Восстания,от2950₽/сутки,вчера,https://spb.cian.ru/rent/flat/284588464/,284588464,Компания LetoPlace приветствует вас!\n\n- Уютн...,Санкт-Петербург,р-н Центральный,Смольнинское,м. Площадь Восстания,8-я Советская улица,16/25,,,,[https://cdn-p.cian.site/images/36/998/551/pos...
164,1.682547e+09,Апартаменты у Кремля!,1200000₽/мес.,шесть дней назад,https://www.cian.ru/rent/flat/285274355/,285274355,"2-комн. апартаменты, 171 м²\nЦЕНА ДЕЙСТВИТЕЛЬН...",Москва,ЦАО,р-н Тверской,м. Охотный ряд,улица Охотный Ряд,2,,,,[https://cdn-p.cian.site/images/09/175/061/kva...
165,1.682547e+09,По соседству с Эрмитажем,от3500₽/сутки,10 часов назад,https://spb.cian.ru/rent/flat/203687192/,203687192,Сдам свою чудесную квартиру всем гостям приезж...,Санкт-Петербург,р-н Центральный,Дворцовый,м. Адмиралтейская,набережная Реки Мойки,18,,,,[https://cdn-p.cian.site/images/5/132/756/kvar...
166,1.682547e+09,Видовая кв у Исаакиевского собора,от3300₽/сутки,10 часов назад,https://spb.cian.ru/rent/flat/206752605/,206752605,Сдам свою чудесную квартиру всем гостям приезж...,Санкт-Петербург,р-н Адмиралтейский,Адмиралтейский,м. Адмиралтейская,Вознесенский проспект,3-5,,,,[https://cdn-p.cian.site/images/1/561/186/kvar...


In [1]:
import pathlib

In [2]:
raw_data = pathlib.Path('/home/charubaiel/workflow/projects/MLOpsini/data/raw/hehe')

In [4]:
raw_data.parent.exists()

False

In [6]:
raw_data.parent.mkdir(parents=True, exist_ok=True)

In [4]:
[i for i in raw_data.glob('*')]

[PosixPath('/home/charubaiel/workflow/projects/MLOpsini/data/raw/42bbae74e56ee3da88c01194874f0530'),
 PosixPath('/home/charubaiel/workflow/projects/MLOpsini/data/raw/5f4c25f60c7e3ab28ea5a3c8c8d596ce')]